In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.losses import CategoricalFocalCrossentropy
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report

# Disable GPU if needed
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Enable Mixed Precision for Faster Training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Paths
dataset_base_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/dataset split/OCTID_Splits"
logs_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientNet/octid/training"
models_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientNet/octid/models"
results_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/5CV/efficientNet/octid/results"

# Hyperparameters
image_size = 224
batch_size = 32
num_classes = 5
epochs = 20

# Prepare results storage
all_results = []

for i in range(1, 6):
    print(f"\n========== Training on Fold {i} ==========")
    
    set_path = os.path.join(dataset_base_path, f"OCTID_Set_{i}")
    train_dir, val_dir, test_dir = [os.path.join(set_path, x) for x in ["train", "validation", "test"]]
    
    # Load datasets
    train_data = tf.keras.utils.image_dataset_from_directory(train_dir, label_mode='categorical', batch_size=batch_size, image_size=(image_size, image_size))
    val_data = tf.keras.utils.image_dataset_from_directory(val_dir, label_mode='categorical', batch_size=batch_size, image_size=(image_size, image_size))
    test_data = tf.keras.utils.image_dataset_from_directory(test_dir, label_mode='categorical', batch_size=batch_size, image_size=(image_size, image_size), shuffle=False)
    
    # Compute Class Weights
    labels = np.concatenate([y.numpy() for _, y in train_data])
    class_weights = compute_class_weight('balanced', classes=np.arange(num_classes), y=np.argmax(labels, axis=1))
    class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
    
    # Load Pretrained EfficientNet
    base_model = EfficientNetV2B0(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))
    base_model.trainable = False
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax")
    ])
    
    # Compile Model
    model.compile(optimizer=Adam(learning_rate=3e-4), loss=CategoricalFocalCrossentropy(from_logits=False), metrics=["accuracy"])
    
    # Callbacks
    model_save_path = os.path.join(models_path, f"efficientNet_OCTID_best_fold_{i}.keras")
    callbacks = [
        ModelCheckpoint(model_save_path, monitor='val_loss', save_best_only=True, verbose=1),
        EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
        CSVLogger(os.path.join(logs_path, f"efficientNet_OCTID_fold_{i}.csv"), append=True)
    ]
    
    # Train Model
    model.fit(train_data, validation_data=val_data, epochs=epochs, class_weight=class_weight_dict, callbacks=callbacks)
    
    # Evaluate on Test Set
    test_loss, test_accuracy = model.evaluate(test_data, verbose=1)
    
    # Predictions
    predictions = model.predict(test_data)
    predicted_classes = np.argmax(predictions, axis=1)
    true_labels = np.concatenate([y.numpy() for _, y in test_data])
    
    # Compute Metrics
    cm = confusion_matrix(np.argmax(true_labels, axis=1), predicted_classes)
    report = classification_report(np.argmax(true_labels, axis=1), predicted_classes, target_names=train_data.class_names, output_dict=True)
    
    # Store Results
    all_results.append({
        "Fold": i,
        "Test Accuracy": test_accuracy,
        "Test Loss": test_loss,
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"]
    })
    
    # Plot Confusion Matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=train_data.class_names, yticklabels=train_data.class_names)
    plt.xlabel("Predicted Class")
    plt.ylabel("Actual Class")
    plt.title(f"Confusion Matrix - EfficientNet OCTID (Fold {i})")
    plt.show()

# Save All Results to Excel
results_df = pd.DataFrame(all_results)
results_excel_file = os.path.join(results_path, "efficientNet_OCTID_5CV_results.xlsx")
results_df.to_excel(results_excel_file, index=False)
print(f"✅ All results saved to {results_excel_file}")
